In [9]:
%pip install pymysql
%pip install mysql-connector

Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 905.8 kB/s eta 0:00:000:0100:01m
  Preparing metadata (setup.py) ... done
  Created wheel for mysql-connector: filename=mysql_connector-2.2.9-cp310-cp310-linux_x86_64.whl size=247951 sha256=c43c2eb0a05bfa57d9db1f84d471cb4388d0326c9daf324a438df198a0dcc55b
  Stored in directory: /home/jovyan/.cache/pip/wheels/76/48/9b/da67ff1a18fe8e9d428f9b1a177716d4a7d363d2bbe83bf6cf
Successfully built mysql-connector
Note: you may need to restart the kernel to use updated packages.


In [10]:
import pandas as pd
import mysql.connector
from sqlalchemy import create_engine

In [15]:
engine = create_engine('mysql+mysqlconnector://root:asutp@db/asutp', echo=False)

data = pd.read_sql("SELECT * FROM `Events`", engine)

data.to_sql(name='Events', con=engine, if_exists='append', index=False)

OperationalError: (mysql.connector.errors.OperationalError) 2055: Lost connection to MySQL server at 'db:3306', system error: 104 Connection reset by peer
[SQL: INSERT INTO `Events` (id, date_created, object, severity, message, nps, file_name) VALUES (%(id)s, %(date_created)s, %(object)s, %(severity)s, %(message)s, %(nps)s, %(file_name)s)]
[parameters: ({'id': 1, 'date_created': datetime.datetime(2022, 5, 21, 1, 15, 38), 'object': 'ПОЛЬЗОВАТЕЛЬ "ДРОЖАЛИН А', 'severity': 0, 'message': 'ПОЛЬЗОВАТЕЛЬ "ДРОЖАЛИН А.С." ГРУППЫ "АДМИНИСТРАТОРЫ" ВОШЁЛ В СИСТЕМУ [АРМ ПТ ОСНОВНОЙ/ДРОЖАЛИН А.С.]', 'nps': 'NPS_1', 'file_name': './storage_example/13. НПС-1/05. Май/2022.05.20 АСУПТ.xlsx'}, {'id': 2, 'date_created': datetime.datetime(2022, 5, 21, 1, 12, 35), 'object': 'ПОЛЬЗОВАТЕЛЬ "КОЛЕСОВ С', 'severity': 0, 'message': 'ПОЛЬЗОВАТЕЛЬ "КОЛЕСОВ С.П." ГРУППЫ "ОПЕРАТОРЫ" ВОШЁЛ В СИСТЕМУ [АРМ ПТ РЕЗЕРВНЫЙ/КОЛЕСОВ С.П.]', 'nps': 'NPS_1', 'file_name': './storage_example/13. НПС-1/05. Май/2022.05.20 АСУПТ.xls